In [1]:
import pandas as pd
import numpy as np
import os
from data.contracts import contract_addresses, stkAAVE_contract_addresses, xSUSHI_contract_addresses

transfers_cols = ['DateTime','From','To','Quantity','Method']

In [2]:
## aUSDC Aave interest bearing USDC
# Holders, filter out contracts
aUSDC_holders = pd.read_csv('data/aUSDC/aUSDC_holders.csv').sort_values(by='Balance', ascending=False)
aUSDC_holders = aUSDC_holders[~aUSDC_holders['HolderAddress'].isin(contract_addresses)].drop([1259]).set_index('HolderAddress')
aUSDC_holders = pd.cut(aUSDC_holders['Balance'], bins=[0,300,3000,30000,300000,3000000,30000000,300000000], labels=[1,2,3,4,5,6,7]).to_frame().rename(columns={'Balance': 'aUSDC_balance'}).astype('float') # Discretize


# Transfers, filter out contracts, keep only transactions that mint/burn aUSDC
aUSDC_transfers = pd.DataFrame()
for f in os.listdir('data/aUSDC/'):
    if f.endswith('.csv') and (f[0] in '0123'):
        data = pd.read_csv('data/aUSDC/'+f, usecols=transfers_cols)
        if data.loc[0]['DateTime'].startswith('2021'):
            data['DateTime'] = pd.to_datetime(data['DateTime'], format='%Y-%m-%d %H:%M:%S')
        else:
            data['DateTime'] = pd.to_datetime(data['DateTime'], format='%d/%m/%Y %H:%M')
        if len(aUSDC_transfers):
            aUSDC_transfers = pd.concat([aUSDC_transfers,data])
        else:
            aUSDC_transfers = data
            
flash_loan_addresses = list(set(list(aUSDC_transfers[aUSDC_transfers['Method']=='Flash Loan']['From'].unique()) + list(aUSDC_transfers[aUSDC_transfers['Method']=='Flash Loan']['To'].unique())))
flash_loan_addresses.remove('0x0000000000000000000000000000000000000000')
aUSDC_transfers = aUSDC_transfers[(~aUSDC_transfers['From'].isin(contract_addresses))&(~aUSDC_transfers['To'].isin(contract_addresses))]
aUSDC_transfers = aUSDC_transfers[(aUSDC_transfers['From']=='0x0000000000000000000000000000000000000000')|(aUSDC_transfers['To']=='0x0000000000000000000000000000000000000000')].reset_index(drop=True)
aUSDC_transfers['Quantity'] = aUSDC_transfers['Quantity'].apply(lambda x: float(''.join(x.split(','))))


# Agg
from__ = aUSDC_transfers[aUSDC_transfers['From']!='0x0000000000000000000000000000000000000000'].rename(columns={"From": "Address"}).drop(columns='To')
to__ = aUSDC_transfers[aUSDC_transfers['To']!='0x0000000000000000000000000000000000000000'].rename(columns={"To": "Address"}).drop(columns='From')
combined_aUSDC_transfers = pd.concat([from__, to__])
latest_day = combined_aUSDC_transfers['DateTime'].max()
# 30-day number of transactions
combined_aUSDC_transfers_30 = combined_aUSDC_transfers[combined_aUSDC_transfers['DateTime']>=(latest_day-pd.Timedelta(days=30))]
num_transactions_30 = np.log10(combined_aUSDC_transfers_30['Address'].value_counts()).to_frame().rename(columns={'Address': '30_day_log10_num_transactions'})
# 90-day number of transactions
combined_aUSDC_transfers_90 = combined_aUSDC_transfers[combined_aUSDC_transfers['DateTime']>=(latest_day-pd.Timedelta(days=90))]
num_transactions_90 = np.log10(combined_aUSDC_transfers_90['Address'].value_counts()).to_frame().rename(columns={'Address': '90_day_log10_num_transactions'})
# 180-day number of transactions
combined_aUSDC_transfers_180 = combined_aUSDC_transfers[combined_aUSDC_transfers['DateTime']>=(latest_day-pd.Timedelta(days=180))]
num_transactions_180 = np.log10(combined_aUSDC_transfers_180['Address'].value_counts()).to_frame().rename(columns={'Address': '180_day_log10_num_transactions'})
# 180-day average size of transaction
avg_size_180 = np.log10(combined_aUSDC_transfers_180[['Address','Quantity']].groupby('Address').aggregate('mean')).rename(columns={'Quantity': 'log10_avg_size'})


# Feature engineering
to_ = pd.DataFrame(aUSDC_transfers['To'].value_counts())
from_ = pd.DataFrame(aUSDC_transfers['From'].value_counts())
features = to_.join(from_, how='outer').rename(columns={"To": "num_USDC_deposits", "From": "num_USDC_withdrawals"}).fillna(0).drop(labels='0x0000000000000000000000000000000000000000')
features['flash_loan'] = [True if i in flash_loan_addresses else False for i in features.index]
features = features.join([aUSDC_holders,num_transactions_30,num_transactions_90,num_transactions_180,avg_size_180], how='left').fillna(0)

In [3]:
## stkAAVE
# Holders, filter out contracts
stkAAVE_holders = pd.read_csv('data/stkAAVE/stkAAVE_holders.csv').sort_values(by='Balance', ascending=False)
stkAAVE_holders = stkAAVE_holders[~stkAAVE_holders['HolderAddress'].isin(stkAAVE_contract_addresses)].set_index('HolderAddress')
stkAAVE_holders = pd.cut(stkAAVE_holders['Balance'], bins=[0,10,100,1000,10000,100000,1000000], labels=[1,2,3,4,5,6]).to_frame().rename(columns={'Balance': 'stkAAVE_balance'}).astype('float') # Discretize

features = features.join([stkAAVE_holders], how='left').fillna(0)
features['stkAAVE'] = features['stkAAVE_balance'].apply(lambda x: True if x else False)

In [4]:
## xSUSHI
# Holders, filter out contracts
xSUSHI_holders = pd.read_csv('data/xSUSHI/xSUSHI_holders.csv').sort_values(by='Balance', ascending=False)
xSUSHI_holders = xSUSHI_holders[~xSUSHI_holders['HolderAddress'].isin(xSUSHI_contract_addresses)].set_index('HolderAddress')
xSUSHI_holders = pd.cut(xSUSHI_holders['Balance'], bins=[0,30,300,3000,30000,300000,3000000,30000000], labels=[1,2,3,4,5,6,7]).to_frame().rename(columns={'Balance': 'xSUSHI_balance'}).astype('float') # Discretize

features = features.join([xSUSHI_holders], how='left').fillna(0)
features['xSUSHI'] = features['xSUSHI_balance'].apply(lambda x: True if x else False)

In [8]:
features

,num_USDC_deposits,num_USDC_withdrawals,flash_loan,aUSDC_balance,30_day_log10_num_transactions,90_day_log10_num_transactions,180_day_log10_num_transactions,log10_avg_size,stkAAVE_balance,stkAAVE,xSUSHI_balance,xSUSHI
0x00000000000cd56832ce5dfbcbff02e7ec639bc9,2.0,2.0,False,0.0,0.00000,0.60206,0.60206,5.005052,0.0,False,0.0,False
0x000000003ce0cf2c037493b1dc087204bd7f713e,1.0,0.0,False,1.0,0.00000,0.00000,0.00000,4.305872,0.0,False,0.0,False
0x0000000484f2217f1a64eb6d24b5cee446faeae5,1.0,1.0,False,0.0,0.00000,0.00000,0.30103,6.024089,0.0,False,0.0,False
0x000000aaee6a496aaf7b7452518781786313400f,0.0,4.0,False,1.0,0.00000,0.00000,0.60206,5.016485,0.0,False,0.0,False
0x000018bbb8df8de9e3eaf772db1c4eec228ef06c,1.0,0.0,False,1.0,0.00000,0.00000,0.00000,1.477355,0.0,False,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
0xffda397f3de16ec7dad3553c220a3b2b292e0255,0.0,1.0,False,0.0,0.00000,0.00000,0.00000,3.538582,0.0,False,0.0,False
0xffdb8f98cfac992824dfac77c88d2107c6df3ab5,1.0,1.0,False,0.0,0.30103,0.30103,0.30103,3.759848,1.0,True,0.0,False
0xffeb122a31cd9bba428adcc1427c33dfca3aeb10,1.0,1.0,False,0.0,0.00000,0.00000,0.30103,4.528979,0.0,False,0.0,False
0xffefdcfff613c9bbb9928f6ff44f07c7b562bfdf,1.0,1.0,False,0.0,0.00000,0.30103,0.30103,4.097385,0.0,False,0.0,False


debt tokens
aave governance
farmed tokens (long-term investor)
